In [18]:
# Imports
import base64
import io
import zlib
from datetime import datetime

In [21]:
def decode_grib_file(encoded_data, output_path):
    """
    Decodes a base64 encoded and zlib compressed GRIB file and saves it to a specified location.

    Args:
    - encoded_data (str): The base64 encoded string of the zlib compressed GRIB file content.
    - output_path (str): The path where the decoded GRIB file should be saved.

    Returns:
    - str: Path to the saved file.
    """

    # Split the received data into lines and extract only the message content lines, skipping the order numbers
    lines = encoded_data.strip().split("\n")
    encoded_data_without_nrs = ''.join([lines[i] for i in range(len(lines)) if i % 3 == 1])

    # Decode the base64 encoded data
    zipped_data = base64.b64decode(encoded_data_without_nrs)

    # Decompress the zlib data to obtain the original GRIB file
    # grib_binary = zlib.decompress(zipped_data)
    grib_binary = zipped_data
    
    # Save the decoded data to the specified path
    with open(output_path, 'wb') as file:
        file.write(grib_binary)

    return output_path

In [24]:
# Copy all messages received from the inreach
TEXT_RECIEVED = """
0
R1JJQgAAWgEAABwCYp7/gAJmAAAaAQ4GAAEADAoAAAAVAAAAAAAgAP8AAAMAAgBdwIEZQIAAfQCA2sAfQB9AQAAAAAAAABIOAANFGL3kBx6d4APXAAA3Nzc3
0
1
R1JJQgAAWgEAABwCYp7/gAJmAAAaAQ4GAAEAMAoAAAAVAAAAAAAgAP8AAAMAAgBdwIEZQIAAfQCA2sAfQB9AQAAAAAAAABIIAANFGJu+CERqjgAWogA3Nzc3
1
2
R1JJQgAAWgEAABwCYp7/gCFpAAoaAQ4GAAEADAoAAAAVAAABAAAgAP8AAAMAAgBdwIEZQIAAfQCA2sAfQB9AQAAAAAAAABIOAADCPpIWByxwBtilQAA3Nzc3
2
3
R1JJQgAAWgEAABwCYp7/gCFpAAoaAQ4GAAEAMAoAAAAVAAABAAAgAP8AAAMAAgBdwIEZQIAAfQCA2sAfQB9AQAAAAAAAABIIAADCTqrUCI49AO11PwA3Nzc3
3
4
R1JJQgAAWAEAABwCYp7/gCJpAAoaAQ4GAAEADAoAAAAVAAABAAAgAP8AAAMAAgBdwIEZQIAAfQCA2sAfQB9AQAAAAAAAABAEAABB4iZgBn2QMwlgNzc3N0dS
4
5
SUIAAFoBAAAcAmKe/4AiaQAKGgEOBgABADAKAAAAFQAAAQAAIAD/AAADAAIAXcCBGUCAAH0AgNrAH0AfQEAAAAAAAAASCAAAwiJNwgg9czoAt38ANzc3Nw==
5
"""

# Define the path where the GRIB file should be saved
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
OUTPUT_PATH = f'./decoded_grib_{current_time}.grb'

# Use the function to decode and save the GRIB file
df = decode_grib_file(TEXT_RECIEVED, OUTPUT_PATH)